In [30]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# BERT Pretrain For PyTorch

## 1. Overview

BERT, or Bidirectional Encoder Representations from Transformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks. This model is based on the [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805) paper. NVIDIA's implementation of BERT is an optimized version of the [Hugging Face implementation](https://github.com/huggingface/pytorch-pretrained-BERT), leveraging mixed precision arithmetic and Tensor Cores on Volta V100 and Ampere A100 GPUs for faster training times while maintaining target accuracy.

## 2. Model architecture
### 2.1Transformer overview
In late 2017, many sequence-to-sequence problems, including machine translation, can be solved with convolutional neural networks or temporal recurrent neural networks (see Figure 1). All the models of that period have in common that they have made extensive use of attention mechanisms to process long sequences. Next,the Transformer architecture introduced in the paper ["Attention is All You Need!"(Vaswani et al., 2017)](https://arxiv.org/abs/1706.03762) provides a new approach that requires only attention mechanism can achieve similar performance without applying convolution or time looping mechanisms at all.
After that, the Transformer became the basis for many models. Not only can these models be used to handle a range of natural language processing (NLP) tasks, but they are now underpinning new waves of progress in many other fields, such as computer vision (for details, see: ["An Image is Worth 16x16 Words: Transformers] for Image Recognition at Scale"](https://arxiv.org/abs/2010.11929)). Since Transformers are the basis of models like BERT, it's important to understand their inner workings.




<center><figure>
    <img src="images/nlp.png" width="800">
    <figcaption>Figure 1. Brief timeline of major breakthroughs in natural language processing</figcaption>
</figure></center>

Transformer is a competitive sequence-to-sequence (seq2seq) model that can replace temporal recurrent neural networks and convolutional neural networks (RNN/CNN) widely used before 2017/2018 for solving a range of sequence modeling tasks , this is because Transformer can make up for a key shortcoming of RNN and CNN, namely the computational cost of the latter two. In the original paper focusing on translation tasks, the author started training on eight GPUs, and it took only three and a half days to successfully achieve the SOTA (best performance) at that time, which is only the training time required for the replaced model. a small part. The reduction in computational requirements allows the research community to dramatically increase the size of datasets and models used, leading to further success in building architectures such as GPT and BERT.

Most seq2seq models consist of an encoder and a decoder. The encoder takes an input sequence and maps it to some intermediate representation (that is, an n-dimensional vector). The decoder then takes this abstract vector and converts it into an output sequence. Figure 2 shows the encoder-decoder architecture using machine translation as an example of the sequence-to-sequence problem.

<center><figure>
    <img src="images/enc_dec.png" width="400">
    <figcaption>Figure 2. seq2seq model</figcaption>
</figure></center>

The Transformer architecture is also composed of an encoder and a decoder (as shown in Figure 3), but compared with the traditional architecture based on RNN or CNN, the Transformer completely relies on the self-attention mechanism, which can directly provide the information between all words in the sentence. Relationship modeling.

<center><figure>
    <img src="images/Transformer_architecture.png">
    <figcaption>Figure 3. from：<a href="https://arxiv.org/abs/1706.03762">Attention is All You Need</a></figcaption>
</figure></center>

Compared with the sequence-to-sequence model of time loop or convolution, the construction mechanism of Transformer will be relatively simple:
1. Tokenizer
2. Embedding layer
3. Positional Encoding
4. Multi-head attention and multi-head attention with mask mechanism
5. And many basic components (such as feedforward layers)

The Transformer layer, which consists of multi-head attention and a feed-forward mechanism, can be repeated multiple times, allowing us to scale up or down the model. The original Transformer model consists of 6 identical layers in the encoder and 6 identical layers in the decoder.

Before introducing the BERT architecture, let us briefly introduce the above components.

### 2.2 Tokenizer

A fundamental part of the model architecture (and outside the neural network) is the tokenizer. A tokenizer converts raw text into an initial numerical representation of the text consisting of tokens (i.e., subwords). The tokenizer is an important part of the model because it allows the model to cope with the complexity of human language. For example, tokenizers can break down words in agglutinative languages ​​into more manageable components, handle new words or foreign words/special characters not present in the original corpus, and ensure that the model produces compact (as lean as possible) textual representations. Every language has a large number of different tokenizers to choose from. Most Transformer-based architectures use trained tokenizers designed to substantially reduce the length of typical sequences. Tokenizers such as WordPiece (for BERT), SentencePiece (for T5 or RoBerta) also have multiple variants, because they are used in many languages ​​and different professional fields (such as healthcare corpora), and these Tokenizers are also trained differently (choose a different maximum number of tokens, or handle case differently). Unfortunately, we lack the literature to understand the impact of the chosen tokenizer on model behavior, and in most cases the initial implementation of the model still uses the default tokenizer. However, it is relatively easy to retrain and change the tokenizer in the model architecture.

In the next notebook, we will take a more in-depth look at tokenizer options and implementations for BERT models.

### 2.3 Embedding
Next, the tokenized text will be processed by the *embedding* mechanism. The embedding vector is generated by an algorithm that converts the raw data into a numerical representation that the neural network can use. Such numerical representations are often referred to as text embeddings.

Text embeddings are often designed with additional properties to help machine learning algorithms cope with the complexity of text and make text representations more compact (less sparse). For example, [Word2Vec](https://arxiv.org/pdf/1310.4546.pdf), [GloVe](https://nlp.stanford.edu/projects/glove/) or [fastText](https://fasttext .cc/) and other algorithms/libraries generate embedding vectors that capture the semantics of individual words, making words that are related to each other appear closer in the embedding space than unrelated words.

In contrast, Transformer architectures are often used with very simple trainable embedding layers, since the neural network itself has enough power to learn these semantic structures. Therefore, the embedding layer in the Transformer implementation is just a matrix (matrix of size $\mathbf{dictionary length}$ x $\mathbf{dimension d_{model}}$ of the embedding vector) that converts raw tokens into target numerical representations. This embedding matrix is ​​trained as part of an end-to-end optimization process. It is worth noting that in the original Transformer implementation, the input and output embedding layers shared the weights of this matrix, and shared the weights with the linear transformation layer before the softmax operation (see the upper right corner of Figure 3).

Both $\mathbf{dictionary length}$ and $\mathbf{d_{model}}$ are architecture hyperparameters that can be changed. The original Transformer model used $\mathbf{d_{model} = 512}$, but in this lab and NVIDIA's implementation, we will use $\mathbf{d_{model} = 1024}$ for the "Large Transformer" model.

### 2.4 Positional Encoding

The words in a sentence are in order, and language models need to take advantage of this property. Since Transformer models do not contain recurrent or convolutional units, we use positional encoding (PE) to indicate the order of words in the input sequence. The positional encoding has the same dimension as the embedding vector, $\mathbf{d_{model}}$, so we can sum the two (see Figure 3). Doing so allows the model to understand the position of each word in the input text.

In Transformer's original paper, positional encoding is just a combination of sine and cosine functions, and this course implements it from positional encoding in the NVIDIA NeMo library, as shown below. Please perform the following unit, which uses a diagram to illustrate positional encoding. See the optional [Positional Encoding](more_details/014_PositionalEncoding_Detail.ipynb) notebook for more details on this calculation, and we'll provide a demonstration example in a follow-up notebook.

Note that the choice of position encoding below is relatively arbitrary. Fortunately, research on the impact of positional encoding on model performance is emerging, a good example of which is ["Rethinking Positional Encoding in Language Pre-training"](https://arxiv.org/pdf/2006.15595.pdf).

### 2.5 Transformer encoder
Now we focus on the design of the encoder mechanism, as shown in Figure 4. The Transformer paper points out that the encoder and decoder are composed of $\mathbf{N = 6}$ identical layers, for a total of 12 layers. Each of the six encoder layers has two sublayers: the first layer is a multi-head self-attention mechanism; the second layer is a simple position-dependent fully-connected feedforward network.

The purpose of the encoder is to encode the source sentence into a hidden state vector; the decoder uses the last representation of the state vector to predict characters in the target language. A single encoder module (one of six) looks like this:

<center><figure>
    <img src="images/encoder1.png" width="500">
    <figcaption>Figure 4. encoder module</figcaption>
</figure></center>

### 2.6 Attention
The "attention" mechanism is the basis of the Transformer model.

Intuitively, we can think of attention in deep learning as an imitation of the human behavior of viewing visual images. We don't usually look at everything in our field of vision, but focus on only those important features, depending on the context of the picture. Likewise, in language, depending on the context, we focus on certain important words (see the example in Figure 5).

The attention mechanism observes the input sequence and decides at each input time step which other parts of the sequence are important. Attention in deep learning can be interpreted as a vector composed of importance weights. In the example below, we can see that "ball" has strong attention to "tennis" and "playing", but the connection between "tennis" and "dog" is weak.


<center><figure>
    <img src="images/attention1.png" width="600">
    <figcaption>Figure 5. Intuition of Attention Mechanisms</figcaption>
</figure></center>

#### 2.6.1 Self Attention
The attention mechanism in the Transformer architecture consists of three components: Query, Key, and Value. Each of these three components has a weight matrix associated with it, which is trained through the optimization process.

The first step in computing self-attention is to create three vectors from each encoder's input vector (embedding). For each word, a query vector, key vector, and value vector are created by multiplying the input vector by the appropriate weight matrix.

The definition of the attention function is as follows:
<img src="images/attentionfunction.png" width="400">

#### 2.6.2 Multi-head Attention
Are 2 heads better than 1 head? What about 8? One of the ways to optimize self-attention is called "multi-head" attention, whereby the model can focus on different locations or subspaces.

In the Transformer architecture, there are h = 8 parallel attention layers, and this attention layer is what we call the "head". This means that there are 8 versions of self-attention, and they can run simultaneously.
<center><figure>
    <img src="images/multiheadattention.png" width="300">
    <figcaption>Figure 7. Multi-head Attention</figcaption>
</figure></center>

### 2.7 Transformer decoder
The decoder works similarly to the encoder, except that the decoder generates only one word at a time, in order from left to right. The decoder will not only pay attention to other previously generated words, but also the final representation generated by the encoder.

Once the encoding phase is complete, the decoding phase can begin. The structure of the decoder is very similar to that of the encoder. In addition to the two sublayers contained in each encoder layer, the decoder inserts a third sublayer to perform multi-head attention on the output of the encoder stack. Similar to the encoder, we first apply residual connections to each sublayer followed by layer normalization.


<center><figure>
    <img src="images/enc_dec_2.png" width="800">
    <figcaption>Figure 8. Decoder</figcaption>
</figure></center>

You may have noticed the "Multi-Head Attention with Masking" layer in the decoder (Figure 3). The self-attention layer in the decoder should allow a position in the decoder to notice all positions in the decoder from left to right up to and including itself, we need to mask the information flow from right to left in the decoder, to preserve autoregressive properties. Combined with the fact that the output embedding is offset by one position (compared to the input), this masking ensures that predictions for position $i$ can only rely on known outputs at positions less than $i$. In other words, multi-head attention with a mask mechanism prevents future words from being part of the attention.

The purpose of the mask is to have zero similarity between a word and words that appear after ("future") the source word. After removing this information, the model will not be able to use this information and the decoder will only consider the similarity to the preceding word.

### 2.8 Bert Overview

["Attention is All You Need!"(Vaswani et al., 2017)](https://arxiv.org/abs/1706.03762) The Transformer model introduced in the paper is an encoder-decoder architecture that relies on attention mechanisms , originally designed to handle neural machine translation (NMT) tasks.


The BERT model uses the same architecture as the encoder of the Transformer. Input sequences are projected into an embedding space before being fed into the encoder structure. Additionally, positional and segment encodings are added to the embeddings to preserve positional information. The encoder structure is simply a stack of Transformer blocks, which consist of a multi-head attention layer followed by successive stages of feed-forward networks and layer normalization. The multi-head attention layer accomplishes self-attention on multiple input representations.

<center><figure>
    <img src="images/From_Transformer_To_Bert_architecture.png">
    <figcaption>Figure 9 <a href="https://arxiv.org/abs/1706.03762">Attention is All You Need</a></figcaption>
</figure></center>

The architecture of the BERT model is almost identical to the Transformer model that was first introduced in the Attention Is All You Need paper. The main innovation of BERT lies in the pre-training step, where the model is trained on two unsupervised prediction tasks using a large text corpus. Training on these unsupervised tasks produces a generic language model, which can then be quickly fine-tuned to achieve state-of-the-art performance on language processing tasks such as question answering.
The BERT paper reports the results for two configurations of BERT, each corresponding to a unique model size. This implementation provides the same configurations by default, which are described in the table below.  

| Model | Hidden layers | Hidden unit size | Attention heads | Feedforward filter size | Max sequence length | Parameters |  
| ---  | --- | --- | --- | --- | --- | --- |
| BSETBASE | 12 encoder | 768 | 12 | 4 x 768 | 512 | 110M |
| BSETLARGE | 24 encoder | 1024 | 16 | 4 x 1024 | 512 | 330M |



Specifically, the BERT model takes as input two sentences separated by a special token [SEP] and is pretrained with two loss functions:
  - Language model to predict word masking
  - Predict the next sentence

To convert the raw text into a numerical representation, the BERT model uses a subword segmentation and tokenization algorithm called WordPiece.

BERT is often used as an encoder for language models. BERT's pretrained checkpoints can be extended by adding additional task-specific layers (see Figure 10). We can then fine-tune these checkpoints on downstream tasks such as token classification (named entity recognition), text classification, question answering, etc.

<center><figure>
    <img src="images/BERT.PNG">
    <figcaption>Figure 10. <a href="https://arxiv.org/pdf/1810.04805.pdf">BERT: Pre-training of Deep Bidirectional Transformers for
        Language Understanding</a>。</figcaption>
</figure></center>

We focus on BERT Pretrain in this notebook.

## 3. BERT Pretrain Task

### 3.1 Get and preprocess the data
For pre-training BERT, we use the concatenation of Wikipedia (2500M words) as well as BookCorpus (800M words). For Wikipedia, we extract only the text passages and ignore headers, lists, and tables. BERT requires that datasets are structured as a document level corpus rather than a shuffled sentence level corpus because it is critical to extract long contiguous sentences.

The preparation of the pre-training dataset is described in the `bertPrep.py` script found in the `data/` folder. The component steps in the automated scripts to prepare the datasets are as follows:

1. Data download and extract - the dataset is downloaded and extracted.

2. Clean and format - document tags, etc. are removed from the dataset.

3. Sentence segmentation - the corpus text file is processed into separate sentences.

4. Sharding - the sentence segmented corpus file is split into a number of uniformly distributed smaller text documents.

5. `hdf5` file creation - each text file shard is processed by the `create_pretraining_data.py` script to produce a corresponding `hdf5` file. The script generates input data and labels for masked language modeling and sentence prediction tasks for the input text shard.

The tools used for preparing the BookCorpus and Wikipedia datasets can be applied to prepare an arbitrary corpus. The `create_datasets_from_start.sh` script in the `data/` directory applies sentence segmentation, sharding, and `hdf5` file creation given an arbitrary text file containing a document-separated text corpus.

To download, verify, extract the datasets, and create the shards in `.hdf5` format, run:<br>
`/workspace/bert/data/create_datasets_from_start.sh`

The pretraining dataset is 170GB+ and takes 15+ hours to download. The BookCorpus server most of the times get overloaded and also contain broken links resulting in HTTP 403 and 503 errors. Hence, it is recommended to skip downloading BookCorpus data by running:<br>
`/workspace/bert/data/create_datasets_from_start.sh wiki_only` <br>

to save time, we already download and preprocessed the datasets , located in `./data`



In [1]:
!nvidia-smi

Mon Apr 18 09:01:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 431.94       Driver Version: 431.94       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 105... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   55C    P8    N/A /  N/A |     78MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### 3.2 Start pretraining
To run on a single node 8 x A100 32G cards, from within the container, you can follow the bellow steps to run pre-training.<br>
Pre-training is performed using the run_pretraining.py script along with parameters.<br>
it runs a job on a single node that trains the BERT-large model from scratch using Wikipedia datasets as training data using the LAMB optimizer.

In [2]:
from datetime import datetime

Set up parameters we use in training.

In [3]:
train_batch_size = "4096"
learning_rate = "6e-3"
precision = "fp16"
num_gpus = "8"
#Proportion of training to perform linear learning rate warmup for. For example, 0.1 = 10% of training.
warmup_proportion = "0.2843"

# number of training steps to perform.
train_steps = "7038"
save_checkpoint_steps = "200"
resume_training = "false"
create_logfile = "true"
accumulate_gradients = "true"

#Number of update steps to accumulate before performing a backward/update pass.
gradient_accumulation_steps = "128"

#Sets the seed to use for random number generation.
seed = "12439"  

job_name = "bert_lamb_pretraining"

#If set to true, performs allreduce only after the defined number of gradient accumulation steps.
allreduce_post_accumulation = "true"

#If set to true, performs allreduce after gradient accumulation steps in FP16.
allreduce_post_accumulation_fp16 = "true"


train_batch_size_phase2 = "2048"
learning_rate_phase2 = "4e-3"

#Proportion of training to perform linear learning rate warmup for. For example, 0.1 = 10% of training.
warmup_proportion_phase2 = "0.128"
train_steps_phase2 = "1563"

#Number of update steps to accumulate before performing a backward/update pass.
gradient_accumulation_steps_phase2 = "512"
# set datasets dir，Should contain .hdf5 files for the task.
DATA_DIR_PHASE1 = "/workspace/bert/data/hdf5_lower_case_1_seq_len_128_max_pred_20_masked_lm_prob_0.15_random_seed_12345_dupe_factor_5/wikicorpus_en"
BERT_CONFIG = "bert_config.json"
DATA_DIR_PHASE2 = "/workspace/bert/data/hdf5_lower_case_1_seq_len_512_max_pred_80_masked_lm_prob_0.15_random_seed_12345_dupe_factor_5/wikicorpus_en" # change this for other datasets
CODEDIR = "/workspace/bert"

#initial checkpoint to start pretraining from (Usually a BERT pretrained checkpoint)
init_checkpoint = "None"
RESULTS_DIR = CODEDIR+"/results"
CHECKPOINTS_DIR = RESULTS_DIR+"/checkpoints"
!mkdir -p $CHECKPOINTS_DIR

The syntax of the command is incorrect.


Transfer parameters to argument format

In [4]:
PREC=""
if precision == "fp16" :
   PREC = " --fp16 "
elif precision == "fp32": 
   PREC = " "
elif precision == "tf32":
   PREC = " "
else:
   print("Unknown <precision> argument")
   exit()

ACCUMULATE_GRADIENTS=""
if accumulate_gradients == "true":
   ACCUMULATE_GRADIENTS = " --gradient_accumulation_steps=" + gradient_accumulation_steps 


CHECKPOINT=""
if resume_training == "true" :
   CHECKPOINT = " --resume_from_checkpoint "


ALL_REDUCE_POST_ACCUMULATION=""
if allreduce_post_accumulation == "true" :
   ALL_REDUCE_POST_ACCUMULATION = " --allreduce_post_accumulation "


ALL_REDUCE_POST_ACCUMULATION_FP16=""
if allreduce_post_accumulation_fp16 == "true" :
   ALL_REDUCE_POST_ACCUMULATION_FP16 = " --allreduce_post_accumulation_fp16 "


INIT_CHECKPOINT=""
if init_checkpoint != "None" :
   INIT_CHECKPOINT = " --init_checkpoint=$init_checkpoint "


INPUT_DIR = DATA_DIR_PHASE1



#### 3.2.1 Phase 1
Pretraining on samples of sequence length 128 and 20 masked predictions per sequence.

Phase 1: (Maximum sequence length of 128)

- Runs on 8 GPUs 
- Uses a learning rate of 6e-3
- Has FP16 precision enabled
- Runs for 7038 steps, where the first 28.43% (2000) are warm-up steps
- Saves a checkpoint every 200 iterations (keeps only the latest 3 checkpoints) and at the end of training. All checkpoints, and training logs are saved to the /results directory (in the container which can be mounted to a local directory).
- Creates a log file containing all the output

Set up the command for Phase1 training.


In [5]:
CMD = CODEDIR + "/run_pretraining.py"
CMD += " --input_dir=" + DATA_DIR_PHASE1
CMD += " --output_dir=" + CHECKPOINTS_DIR
CMD += " --config_file=" + BERT_CONFIG
CMD += " --bert_model=bert-large-uncased"
CMD += " --train_batch_size=" + train_batch_size
CMD += " --max_seq_length=128"
CMD += " --max_predictions_per_seq=20"
CMD += " --max_steps=" + train_steps
CMD += " --warmup_proportion=" + warmup_proportion
CMD += " --num_steps_per_checkpoint=" + save_checkpoint_steps
CMD += " --learning_rate=" + learning_rate
CMD += " --seed=" + seed
CMD += PREC
CMD += ACCUMULATE_GRADIENTS
CMD += CHECKPOINT
CMD += ALL_REDUCE_POST_ACCUMULATION
CMD += ALL_REDUCE_POST_ACCUMULATION_FP16
CMD += INIT_CHECKPOINT
CMD += " --do_train"
CMD += " --json-summary " + RESULTS_DIR + "/dllogger.json "
CMD = "python3 -m torch.distributed.launch --nproc_per_node=" + num_gpus + " " + CMD

Set up the log file

In [6]:
now = datetime.now()
timestamp1 = datetime.timestamp(now)

LOGFILE1=RESULTS_DIR+"/"+job_name + ".pyt_bert_pretraining_phase1_" + precision + "_" + str(timestamp1) + ".log"



Run Phase1 training

In [7]:
!$CMD |& tee $LOGFILE1

& was unexpected at this time.


### 3.2.2 Phase 2
Pretraining on samples of sequence length 512 and 80 masked predictions per sequence.

Phase 2: (Maximum sequence length of 512)

- Runs on 8 GPUs 
- Uses a learning rate of 4e-3
- Has FP16 precision enabled
- Runs for 1563 steps, where the first 12.8% are warm-up steps
- Saves a checkpoint every 200 iterations (keeps only the latest 3 checkpoints) and at the end of training. All checkpoints, and training logs are saved to the /results directory (in the container which can be mounted to a local directory).
- Creates a log file containing all the output

Set up the command for Phase2 training.

In [8]:
ACCUMULATE_GRADIENTS=""
if accumulate_gradients == "true":
   ACCUMULATE_GRADIENTS = " --gradient_accumulation_steps=" + gradient_accumulation_steps_phase2


INPUT_DIR=DATA_DIR_PHASE2

CMD = CODEDIR + "/run_pretraining.py"
CMD += " --input_dir=" + DATA_DIR_PHASE2
CMD += " --output_dir=" + CHECKPOINTS_DIR
CMD += " --config_file=" + BERT_CONFIG
CMD += " --bert_model=bert-large-uncased"
CMD += " --train_batch_size=" + train_batch_size_phase2
CMD += " --max_seq_length=512"
CMD += " --max_predictions_per_seq=80"
CMD += " --max_steps=" + train_steps_phase2
CMD += " --warmup_proportion=" + warmup_proportion_phase2
CMD += " --num_steps_per_checkpoint=" + save_checkpoint_steps
CMD += " --learning_rate=" + learning_rate_phase2
CMD += " --seed=" + seed
CMD += PREC
CMD += ACCUMULATE_GRADIENTS
CMD += CHECKPOINT
CMD += ALL_REDUCE_POST_ACCUMULATION
CMD += ALL_REDUCE_POST_ACCUMULATION_FP16
CMD += " --do_train --phase2 --resume_from_checkpoint --phase1_end_step=" + train_steps
CMD += " --json-summary " + RESULTS_DIR + "/dllogger.json "
CMD = "python3 -m torch.distributed.launch --nproc_per_node=" + num_gpus + " " + CMD

In [9]:
now = datetime.now()
timestamp2 = datetime.timestamp(now)

LOGFILE2=RESULTS_DIR+"/"+job_name + ".pyt_bert_pretraining_phase2_" + precision + "_" + str(timestamp2) + ".log"

Run Phase1 training

In [10]:
!$CMD |& tee $LOGFILE2

& was unexpected at this time.


## 4.More information
for fine tune guides, inference guides , and performance of them , please refer to our [DeeplearningExample](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/LanguageModeling/BERT) repo in github. 
